<a href="https://colab.research.google.com/github/huyduatop29/Large-Language-Models/blob/main/full_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

In [2]:
!python -m venv .venv

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/.venv/bin/python3



In [3]:
!pip install replicate

In [4]:
!pip install streamlit_option_menu

In [5]:
!npm install -g localtunnel


changed 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [6]:
!pip install python-docx

In [7]:
!pip install pdfplumber

In [8]:
import os
import pdfplumber
from docx import Document
import json
from pdfminer.pdfdocument import PDFSyntaxError

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except PDFSyntaxError as e:
        print(f"Could not read PDF file {pdf_path}: {e}")
        return None  # Trả về None nếu tệp PDF không hợp lệ
    except Exception as e:
        print(f"Unexpected error when reading {pdf_path}: {e}")
        return None
    return text

def extract_text_from_docx(docx_path):
    try:
        doc = Document(docx_path)
        text = ""
        for para in doc.paragraphs:
            text += para.text + "\n"
        return text
    except Exception as e:
        print(f"Could not read DOCX file {docx_path}: {e}")
        return None  # Trả về None nếu tệp DOCX không hợp lệ

def process_folder(folder_path):
    data = {}
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            text = extract_text_from_pdf(file_path)
            if text:  # Chỉ thêm văn bản nếu tệp PDF hợp lệ
                data[filename] = text
            else:
                os.remove(file_path)  # Xóa tệp không hợp lệ
                print(f"Removed invalid PDF file: {filename}")
        elif filename.endswith('.docx'):
            text = extract_text_from_docx(file_path)
            if text:  # Chỉ thêm văn bản nếu tệp DOCX hợp lệ
                data[filename] = text
            else:
                os.remove(file_path)  # Xóa tệp không hợp lệ
                print(f"Removed invalid DOCX file: {filename}")
        else:
            print(f"Unsupported file format: {filename}")
    return data

def save_data_as_json(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# Đường dẫn đến thư mục chứa các tệp PDF và Word
folder_path = '/content/data'

# Xử lý thư mục và trích xuất văn bản
extracted_data = process_folder(folder_path)

# Lưu dữ liệu trích xuất thành tệp JSON
save_data_as_json(extracted_data, 'output_data.json')

print("Dữ liệu đã được lưu vào output_data.json")

Unsupported file format: .ipynb_checkpoints
Dữ liệu đã được lưu vào output_data.json


In [9]:
import json

# Đường dẫn tới file JSON
json_file_path = '/content/output_data.json'

# Tải dữ liệu từ file JSON
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Kiểm tra các tệp và nội dung
print(data.keys())

dict_keys(['Thông-tin-đội-hình (1).docx', 'Tiềm-lực-chính-trị-tinh-thần-là-nhân-tố-cơ-bản-tạo-nên-sức-mạnh-của-quốc-phòng.docx', 'Thông-tin-đội-hình.docx', 'pact1-đttt.docx', 'Tán gái đại cương.docx', 'PHƯƠNG THỨC TỒN TẠI.docx', 'Phao.docx', 'trợ lí ảo AI.docx', 'Triết-học.docx', 'TLCTDLGT.docx', 'triet2cau.docx'])


In [10]:
!pip install -U sentence-transformers

In [11]:
!pip install -q pyvi

In [13]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

# Tải mô hình embedding
embedding_model = SentenceTransformer('dangvantuan/vietnamese-embedding')

# Đọc dữ liệu từ file JSON
json_file_path = 'output_data.json'
try:
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
except Exception as e:
    print(f"Error reading JSON file: {e}")
    data = {}

# Mã hóa các văn bản từ output_data.json
embeddings = {}
for filename, text in data.items():
    try:
        # Tokenize văn bản
        tokenized_text = tokenize(text)
        if not tokenized_text.strip():  # Kiểm tra văn bản sau khi tokenize
            print(f"Skipping empty text in file {filename}")
            continue  # Bỏ qua nếu văn bản rỗng

        # Chia văn bản thành các phần nhỏ nếu quá dài
        sentences = tokenized_text.split('.')
        text_embeddings = []
        for sentence in sentences:
            sentence = sentence.strip()
            if sentence:  # Bỏ qua câu rỗng
                embedding = embedding_model.encode([sentence])[0]
                text_embeddings.append(embedding)

        if text_embeddings:  # Kiểm tra nếu có embedding
            # Tính trung bình của tất cả các embedding câu
            avg_embedding = np.mean(text_embeddings, axis=0).tolist()
            embeddings[filename] = avg_embedding
        else:
            print(f"No valid sentences for file {filename}")
    except Exception as e:
        print(f"Error encoding file {filename}: {e}")
        continue  # Bỏ qua nếu gặp lỗi

# Lưu embeddings vào file JSON
output_file = 'output_embeddings.json'
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(embeddings, f, ensure_ascii=False, indent=4)
    print("Successfully encoded and saved all valid files.")
except Exception as e:
    print(f"Error saving embeddings to JSON file: {e}")


Error encoding file Phao.docx: index out of range in self
Error encoding file TLCTDLGT.docx: index out of range in self
Successfully encoded and saved all valid files.


In [18]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import streamlit as st
from streamlit_option_menu import option_menu
from sklearn.metrics.pairwise import cosine_similarity

# Tải mô hình embedding
embedding_model = SentenceTransformer('dangvantuan/vietnamese-embedding')

# Tải mô hình GPT-Neo và tokenizer
gpt_model = GPTNeoForCausalLM.from_pretrained("NlpHUST/gpt-neo-vi-small")
gpt_tokenizer = GPT2Tokenizer.from_pretrained("NlpHUST/gpt-neo-vi-small")

# Đọc embeddings từ file JSON
output_file = 'output_embeddings.json'
try:
    with open(output_file, 'r', encoding='utf-8') as f:
        embeddings = json.load(f)
except Exception as e:
    st.error(f"Error reading JSON file: {e}")
    embeddings = {}

# Đọc dữ liệu từ file JSON gốc
json_file_path = 'output_data.json'
try:
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
except Exception as e:
    st.error(f"Error reading JSON file: {e}")
    data = {}

with st.sidebar:
    selected = option_menu("Menu", ["Home", 'ChatBot', "Settings"],
        icons=['house', 'chat', 'gear'], menu_icon="cast", default_index=1)
    selected

# Xử lý giao diện người dùng
col1, col2 = st.columns([1, 1])
with col1:
    log = st.button("Login")
with col2:
    sign = st.button("Sign Up")

def sign_up():
    st.markdown("<h1>User Registration</h1>", unsafe_allow_html=True)
    with st.form("form 2"):
        col1, col2 = st.columns([1, 1])
        f_name = col1.text_input("First Name")
        l_name = col2.text_input("Last Name")
        email = st.text_input("Email Address")
        password = st.text_input("Password", type="password")
        s_status = st.form_submit_button("Submit")

        if s_status:
            if f_name == "" or l_name == "":
                st.warning("Please enter your first and last name!")
            else:
                st.success("Sign up successful!")

def login():
    st.markdown("<h1>User Login</h1>", unsafe_allow_html=True)
    with st.form("form 1"):
        st.text_input("Email Address")
        st.text_input("Password", type="password")
        st.form_submit_button("Submit")

if log:
    login()
if sign:
    sign_up()

def get_best_match(question, embeddings, data):
    # Tokenize và mã hóa câu hỏi
    tokenized_question = tokenize(question)
    question_embedding = embedding_model.encode([tokenized_question])[0]

    # Tính toán độ tương đồng cosine giữa câu hỏi và tất cả các embedding
    best_score = -1
    best_match = None
    for filename, embedding in embeddings.items():
        similarity_score = cosine_similarity([question_embedding], [embedding])[0][0]
        if similarity_score > best_score:
            best_score = similarity_score
            best_match = filename

    # Trả về văn bản tương ứng với độ tương đồng cao nhất
    return best_match, best_score, data.get(best_match, "")

# Input text
val = st.text_input("How may I assist you today?")

if st.button("Find Answer"):
    if val:
        # Tìm văn bản phù hợp nhất
        best_match, best_score, answer = get_best_match(val, embeddings, data)

        # Tạo prompt từ văn bản phù hợp nhất
        prompt = answer + "\nCâu hỏi: " + val + "\nTrả lời:"

        # Mã hóa prompt và sinh câu trả lời
        input_ids = gpt_tokenizer(prompt, return_tensors="pt").input_ids
        gen_tokens = gpt_model.generate(input_ids, do_sample=True, temperature=1.0, max_new_tokens=150)
        gen_text = gpt_tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

        st.write(f"Câu trả lời từ tệp {best_match} với độ tương đồng {best_score}:")
        st.write(answer)
        st.write("Câu trả lời của GPT-Neo:")
        st.write(gen_text)
    else:
        st.warning("Please enter a question.")


In [20]:
!python -m streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.169.50:8501

your url is: https://cold-words-turn.loca.lt
  Stopping...
^C
